# Exercise #1 - Getting Started with Spark SQL
In this exercise you will play with some basic SparkSQL functions

## Task 1 - Reading data into a DataFrame
1. Load the following path: `"/databricks-datasets/samples/population-vs-price/data_geo.csv"` into a Dataframe.
2. Print how many rows are in the dataset
3. Drop rows with missing values and count again how many rows are left.
4. Show the first 10 lines of the DataFrame
5. Use databricks's `display(df)` instead of `df.show()` to view this data in a tabular format.

In [3]:
# Write your code here
# 1
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/samples/population-vs-price/data_geo.csv")

# What's the purpose of the "inferSchema" option? Hint: Use df.printSchema() to find out.

# 2
print("There are {} rows".format(df.count()))

# 3
df = df.dropna()
print("There are {} rows after dropping rows with missing values".format(df.count()))

# 4
df.show(10)  # First try this, then the line below

# 5
#display(df)

## Task 2 - Basic SQL on Dataframes
1. Select the `State`, `2014 Population estimate` and `2015 median sales price` fields from the dataframe.
2. Add the condition `where 'State'='Alabama'`.

In [5]:
# Write your code here

# 1 - There are some ways of doing this
# Option 1
df_states = df.select("State", "2014 Population estimate", "2015 median sales price")
df_states = df.select(df["State"], df["2014 Population estimate"], df["2015 median sales price"])
df_states.show(2)

# 2 - Again, There are some ways of doing this. Filter() and Where() have the same functionality.
df_alabama = df_states.where("State = 'Alabama'")
df_alabama = df_states.filter(df.State == 'Alabama')
df_alabama.show(2)

## Task 3 - Simple Transformations
1. Load the file `"/databricks-datasets/iot/iot_devices.json"` into a Dataframe.
2. Filter out all devices whose temperature exceed 25 degrees.
3. Keep the columns: `device_name`, `temp`, `humidity` and `cca3`.
4. Group the rows by `cca3` and compute the average temp and humidity. Rename the last two columns to be: `average_temp` and `average_humidity`.
5. Sort the df from step 4 by humidity, descending.
6. Using the df from step 3, group the rows by `cca3` but this time compute the min temp, max humidity and total count of rows.

In [7]:
# Write your code here
from pyspark.sql import functions as F

# 1
df = spark.read.json("/databricks-datasets/iot/iot_devices.json")

# 2
df = df.filter(df.temp <= 25)

# 3
df = df.select("device_name", "temp", "humidity", "cca3")
df.show(5)

# 4
# Option 1
df_avg = df\
      .groupBy("cca3")\
      .avg()\
      .withColumnRenamed("avg(temp)", "average_temp")\
      .withColumnRenamed("avg(humidity)", "average_humidity")

# Option 2:
df_avg = df\
      .groupBy("cca3")\
      .agg(F.avg("temp").alias("average_temp"), 
           F.avg("humidity").alias("average_humidity"))

# 5
df_avg = df_avg.sort("average_humidity", ascending=False)
df_avg.show(5)

# 6
# Option 1
df_min_max = df\
      .groupBy("cca3")\
      .agg({"temp": "min", "humidity": "max", "*": "count"})

# Option 2
df_min_max = df\
      .groupBy("cca3")\
      .agg(F.min("temp"), F.max("humidity"), F.count("*"))

df_min_max.show(5)

## Task 4 - Pets Dataframe
1. Create a data frame from the data below, it should have a schema with the right types
```
id,breed_id,nickname,birthday,age,color,weight
1,1,"King","2014-11-22 12:30:31",5,"brown",10.0
2,3,"Argus","2016-11-22 10:05:10",10,None,5.5
3,1,"Chewie","2016-11-22 10:05:10",15,None,12.0
3,2,"Maple","2018-11-22 10:05:10",17,"white",3.4
4,2,None,"2019-01-01 10:05:10",13,None,10.0
```

Hint: The `birthday` field should be of type `TimestampType()`. Therefore you'll need to use python's `datetime.strptime` method on the date strings to get datetime objects, for example: `datetime.strptime("2014-11-22 12:30:31", '%Y-%m-%d %H:%M:%S')`

In [9]:
# Write your code here
from pyspark.sql import types as T
from datetime import datetime

schema = T.StructType([
    # Fill this array with all the fields
    T.StructField("id", T.IntegerType(), False),
    T.StructField("breed", T.StringType(), True),
    T.StructField("nickname", T.StringType(), True),
    T.StructField("birthday", T.TimestampType(), True),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("color", T.StringType(), True),
    T.StructField("weight", T.DoubleType(), True),
])

data = [
  # Fill this array with all the data
  (1,1,"King",datetime.strptime("2014-11-22 12:30:31", '%Y-%m-%d %H:%M:%S'),5,"brown",10.0),
  (2,3,"Argus",datetime.strptime("2016-11-22 10:05:10", '%Y-%m-%d %H:%M:%S'),10,None,5.5),
  (3,1,"Chewie",datetime.strptime("2016-11-22 10:05:10", '%Y-%m-%d %H:%M:%S'),15,None,12.0),
  (3,2,"Maple",datetime.strptime("2018-11-22 10:05:10", '%Y-%m-%d %H:%M:%S'),17,"white",3.4),
  (4,2,None,datetime.strptime("2019-01-01 10:05:10", '%Y-%m-%d %H:%M:%S'),13,None,10.0),
]

# df_pets = # Write your code here
df_pets = spark.createDataFrame( 
    data=data,
    schema=schema
)

df_pets.show()

## Task 5 - Manipulations on the Pets Dataframe
For the following steps, use the `df_pets` created on the previous task
1. Create a new Dataframe `df_pets2` by changing/adding the following columns:
  1. Add `birthday_date` - a new column of DateType type, casted from the `birthday` column.
  2. Add `owned_by` - a new column with the literal `'me'`. (In other words, the word `'me'` will appear in every row).
  3. Rename the `pet` column to `pet_id`.
2. Filter the DF such that only pets born after `January 1st, 2017` are on the table.

In [11]:
# Write your code here
from pyspark.sql import functions as F

# 1
df_pets2 = df_pets\
            .withColumn('birthday_date', F.col('birthday').cast('date'))\
            .withColumn('owned_by', F.lit('me'))\
            .withColumnRenamed('id', 'pet_id')

# 2
df_pets2 = df_pets2.where(F.col('birthday_date') > datetime(2017,1,1))
df_pets2.show()

## Task 6 - Array, Map and Fill
For the following steps, create a new df and use the `df_pets` created on task 4
1. Fill rows with missing `nickname` and `color` with a default value `Unknown Name` and `Unknow Color` respectively. Hint: Use `df.fillna({...})`.
2. Add a new column `double_age` that is the result of multiplying the pet's age by 2.
3. Add a new column `array_column` to the df. It should contain an array with the literal `"array"`, `today's datetime`, the `breed` and the `double_age` column.
4. Add a new column `mapping`. It should be a map with the literal `AGE` as key and `age` as value. Hint: Use `F.create_map().`
5. Add a new column `math_operation`. You should take the 4th element in the array inside the `array_column` (the `double age`), minus the value of the mapping in column `mapping` (the `age`) and multiply the result by the `id`. The result should be of type `int`.

In [13]:
# Write your code here
from pyspark.sql import functions as F
# 1
df = df_pets\
  .fillna({
      'nickname': 'Unknown Name',
      'color':      'Unknown Color',
    })

# 2
df = df.withColumn('double_age', df.age * 2)

# 3
df = df\
      .withColumn('array_column', F.array([
        F.lit("array"),
        F.lit(datetime.now()),
        df.breed,
        df.double_age
      ]))

# 4
df = df.withColumn("mapping", F.create_map(F.lit("AGE"), "age"))

# 5
df.withColumn("math_operation", ((df.array_column.getItem(3)-df.mapping.AGE)*df.id).cast('int')).show()
# df.show()

## Task 7 - UDF
Sometimes you can't find functions that can perform what you want. For this situations you might want to implement your own UDF (User Defined Function).

These are functions written in python code that take a subset of columns as the input and returns a new column back.

Your task is to write a UDF that will convert all chars in a given string to UPPERCASE and will replace every 's' with a '$', every 'a' with a '@' adn every i with a '!'.

1. Write a method in python that receives a string. It should convert all its chars to `UPPERCASE` and replace every `'S'` with a `'$'`, every `'A'` with a `'@'`, every `'I'` with a `'!'` and every `'O'` with `'0'`.
2. Register the UDF as a spark udf. It should return a column of type `StringType()`.
3. Apply the UDF to the columns `nickname` and `color` from the `df_pets` built on task 4. Apply it another time to the `nickname` column but with `max_length=2`. Hint: Use a literal to send the `max_length` param.

In [15]:
# Write your code here
from pyspark.sql import types as T
from pyspark.sql import functions as F

# 1
def uppercase_and_replace(word, max_length=10):
  return word.upper().replace('S', '$').replace('A', '@').replace('I', '!').replace('O', '0')[:min(len(word),max_length)] if word else None

# 2
udf_upper_and_replace = F.udf(uppercase_and_replace, T.StringType())

# 3
df_pets\
  .withColumn('nickname_upper', udf_upper_and_replace(F.col('nickname')))\
  .withColumn('color_upper', udf_upper_and_replace("color"))\
  .withColumn('color_upper_trimmed', udf_upper_and_replace("color", F.lit(2)))\
  .show()

# Option 2: Less Steps using a Decorator
# from pyspark.sql.functions import udf
# @udf('string', 'int')  # The input types
# def uppercase_and_replace(word, max_length=10):
#   return word.upper().replace('S', '$').replace('A', '@').replace('I', '!').replace('O', '0')[:min(len(word),max_length)] if word else None

## Task 8 - DataFrame to RDD
Sometimes you will need to convert a Dataframe into an RDD to do some extra manipulations that can't be done with SQL functions.

In this task, you'll have to:
1. Convert the Pets Dataframe created in Task 4 into an RDD.
2. Extract the `'age'` column from each `Row`. Hint: Values can be extracted from a `Row` object like you extract values from a Python Dict. i.e: `Row(a=1, b=2, c=3)['a'] returns '2'`.
3. Multiply the age value by itself with a `map` function.
4. Use the `reduce` function to sum the values from the previous step.

In [17]:
# Write your code here
from operator import add

rdd_pets = df_pets.rdd
rdd_pets\
      .map(lambda r: r['age'])\
      .map(lambda v: v*v)\
      .reduce(add)

## Task 9 - RDD to DataFrame

Sometimes you'll also need to convert an RDD into a DataFrame.

Below you are given an RDD. Your job is to convert it into a DataFrame with the following schema:

`id, breed_id, nickname, birthday, age, color, weight`

In [19]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

rdd = sc.parallelize(
        ['1,1,"King","2014-11-22 12:30:31",5,"brown",10.0'
        ,'2,3,"Argus","2016-11-22 10:05:10",10,"black",5.5'
        ,'3,1,"Chewie","2016-11-22 10:05:10",15,"green",12.0'
        ,'3,2,"Maple","2018-11-22 10:05:10",17,"white",3.4'
        ,'4,2,"Coby","2019-01-01 10:05:10",13,"yellow",10.0']
)

# Write your code here

# Option 1
df_pets_from_rdd = rdd\
                    .map(lambda s: s.split(','))\
                    .toDF(["id", "breed_id", "nickname", "birthday", "age", "color", "weight"])

# Option 2
from pyspark.sql import Row
df_pets_from_rdd = rdd\
                    .map(lambda s: s.split(','))\
                    .map(lambda l: Row(id=l[0], breed_id=l[1], nickname=l[2], birthday=l[3], age=l[4], color=l[5], weight=l[6]))\
                    .toDF()

df_pets_from_rdd.show()

## Run SQL queries
In PySpark, you can issues SQL queries instead of using SparkSQL.

To do that you must register the table so it is accesible via SQL Context and then a query can be executed, let's walk over the following example:

In Task 1, you loaded `"/databricks-datasets/samples/population-vs-price/data_geo.csv"` into a Dataframe. After doing that, you'll need to register it:

In [22]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/samples/population-vs-price/data_geo.csv")

df.createOrReplaceTempView("data_geo")

You can now query the table by using sql queries, the first option is using spark:

In [24]:
spark.sql("select `State Code`, `2015 median sales price` from data_geo").show(5)

In databricks you can also issue sql queries by defining the cell as an SQL one, just write `%sql` at the top of the cell and then the query.

An additional benefit of using the Databricks is that you can quickly view this data with a number of embedded visualizations. Click the down arrow next to the Chart Button to display a list of visualization types.
Then, select the Map icon to create a map visualization of the sale price SQL query below:

In [26]:
%sql
select `State Code`, `2015 median sales price` from data_geo
-- Use the 'Map' visualization

In [27]:
%sql
select City, `2014 Population estimate` from data_geo where `State Code` = 'WA' limit 3;
-- Use the 'Bar' visualization

In [28]:
%sql
select City, `2014 Population estimate`/1000 as `2014 Population Estimate (1000s)`, `2015 median sales price` as `2015 Median Sales Price (1000s)` from data_geo order by `2015 median sales price` desc limit 10;
-- Use the 'Pie' visualization, the Click on the 'Plot Options' and select `2015 Median Sales Price (1000s)` and `2014 Population Estimate (1000s)` as the values. Enlarge the graph.